In [ ]:
import pdfplumber,re,pandas as pd, os,re

def load_pdf(path, dont_find_table = True):
    pdf = pdfplumber.open(path)
    print(pdf.metadata)
    print('Total Pages: ', len(pdf.pages))
    if dont_find_table:
        return pdf
    for i, page in enumerate(pdf.pages):
        tb_found = page.find_tables()
        if len(tb_found) > 0:
            print('Page Index', i, 'Has ', len(tb_found), ' Table.')
    return pdf

def extract_table(pages):
    merged_tb = []
    for page in pages:
        tb = page.extract_table(
            table_settings={"horizontal_strategy": "lines"})
        if tb is None:
            continue
        merged_tb = merged_tb + tb
    return merged_tb

def save_to_xlsx(table, path):
    table_data = pd.DataFrame(table[1:], columns = table[0])
    table_data.to_excel(path, index = False)
    
home = './ICPC/'
output = './extracted/ICPC/'
file = '46ICPC沈阳.pdf'
path = home + file
output_path = output + file.replace('.pdf', '.xlsx')

pdf = load_pdf(path, False)


In [ ]:
table = extract_table(pdf.pages[10:36])
table

In [ ]:
refined = []
title = ['学校名称', '队伍名称', '队员1姓名', '队员2姓名', '队员3姓名', '队伍类型']
cn_team_index = 1
column_mapping = {
}
cn_col = [0, 1, 2, 3, 4, 5]
en_col = []
row_skip = [[]]
row_replace = {}
school_fix = {}
refined.append(title)
print(title)

def try_fix(data, add):
    print('--- Fixing Starts ---')
    print('Data To Fix: ', data)
    print('Data To Add: ', add)
    global title
    global column_mapping
    for mapper in column_mapping:
        if len(add) > mapper and add[mapper] is not None and len(add[mapper]) > 0:
            to = column_mapping[mapper]
            data[to] = data[to] + add[mapper]
            add[mapper] = None
    print('Fixed Data: ', data)
    print('--- Fixing Ends ---')
    fix_succeed = True
    for i, item in enumerate(add):
        if item is not None and len(item) > 0:
            fix_succeed = False
            print('Cannot Fix Data: ',i, item)
    if not fix_succeed:
        raise Exception('Unprocessed Data: ', add)
    return data
    

for raw in table[1:]:
    baked = []
    for data in raw:
        if data is not None and len(data) > 0:
            baked.append(data)

    if baked in row_skip:
        continue

    if baked[0] in row_replace:
        baked = row_replace[baked[0]].copy()
    
    if len(baked) != len(title):
        refined[-1] = try_fix(refined[-1].copy(), raw)
        continue

    for i in cn_col:
        baked[i] = baked[i].replace('\n', '')
    for i in en_col:
        baked[i] = baked[i].replace('\n', ' ')
    
    if baked == title:
        continue

    if baked[1] in school_fix and int(baked[0]) in school_fix[baked[1]][1]:
        baked[1] = school_fix[baked[1]][0]
    print(baked)
    if baked[cn_team_index][0] == '☆':
        baked[cn_team_index] = baked[cn_team_index][1:]
    refined.append(baked)
refined

In [ ]:
#['中原工学院', '队名起个啥', '赵孜硕', '叶宇恒', '徐伟明', '正式']
id = 1
results = []
for row in refined[1:]:
    results.append([id, row[0], '', row[1]] + row[2:5]+ ['', '', '', 'Shadow' if "打星" in row[-1] else 'Primary'])
    id = id + 1
    if len(results[-1]) != 11:
        print(results[-1])
#results

In [ ]:
schema = ['序号', '学校名称', '英文队名', '中文队名', '队员1', '队员2', '队员3', '教练', '区域', '座位', '备注']
teams = pd.DataFrame(results, columns = schema)
teams

In [ ]:
teams.to_excel(output_path, index = False)